In [5]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK
import numpy as np
from scipy.optimize import minimize

References

* https://apmonitor.com/che263/index.php/Main/PythonOptimization (initial formulation)
* https://stackoverflow.com/questions/43702352/maximize-optimization-using-scipy (-1 trick)
* https://het.as.utexas.edu/HET/Software/Scipy/generated/scipy.optimize.minimize.html ((0,None) trick)
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize API Ref
* https://stackoverflow.com/questions/17009774/quadratic-program-qp-solver-that-only-depends-on-numpy-scipy Plot
* https://realpython.com/python-scipy-cluster-optimize/ Walkthrough 

# 1.  
An engineer at Fertilizer Company has synthesized a sensational new fertilizer made of just two interchangeable 
basic raw materials.  The company wants to take advantage of this opportunity and produce as much as possible of 
the new fertilizer.  The company currently has \\$40,000 to buy raw materials at a unit price of \\$8000 and \\$5000 per unit, respectively.  When amounts x1 and x2 of the basic raw materials are combined, a quantity q of fertilizer results given by:  𝑞=4𝑥1 +2𝑥2 −0.5𝑥12 −0.25𝑥22 
 
## Part A:    
Formulate as a constrained nonlinear program.  Clearly indicate the variables, objective function, and 
constraints.   
 
## Part B:    
Solve the Program (provide exact values for all variables and the optimal objective function). 

In [14]:
def objective(x):
    '''
    When amounts x1 and x2 of the basic raw materials are combined, a quantity q of fertilizer results
    If this were a min problem, it would simply be q = 4*x1 + 2*x2 - 0.5*x1**2 - 0.25*x2**2
    Since this is a max, we multiply it by -1
    '''
    x1 = x[0]
    x2 = x[1]
    q = -1 * (4*x1 + 2*x2 - 0.5*x1**2 - 0.25*x2**2)
    return q

def constraint1(x):
    '''
    The company currently has $40,000 to buy raw materials at a unit price of $8000 and $5000 per unit, respectively
    '''
    x1 = x[0]
    x2 = x[1]
    return 8000*x1 + 5000*x2 - 40000


# initial guesses
x0 = [0, 0]

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
b = (0.0,None)
bnds = (b, b) ## greater than 0 constraints
con1 = {'type': 'ineq', 'fun': constraint1}
cons = ([con1])
solution = minimize(objective,x0,method='SLSQP', bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))

Initial Objective: -0.0
Final Objective: -11.999999998793982
Solution
x1 = 4.000013055847088
x2 = 4.000066956398335


# 2.  

The area of a triangle with sides of length a, b, and c is √𝑠(𝑠−𝑎)(𝑠−𝑏)(𝑠−𝑐), where s is half the perimeter 
of the triangle.  We have 60 feet of fence and want to fence a triangular-shaped area.   

## Part A:     
Formulate the problem as a constrained nonlinear program that will enable us to maximize the area of the 
fenced area, with constraints.  Clearly indicate the variables, objective function, and constraints. 
 
Hint: The length of a side of a triangle must be less than or equal to the sum of the lengths of the other two sides. 
 
## Part B: 
Solve the Program (provide exact values for all variables and the optimal objective function). 


In [27]:
def triangle_area(x, *args):
    '''
    a, b, and c are the side of a triangle
    s is half the perimeter of the triangle
    '''
    a, b, c = x
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))
    return -area

def triangle_constraint(x):
    a, b, c = x
    return [60 - (a + b + c), a + b - c, a + c - b, b + c - a, a, b, c]

x0 = [10, 10, 10] # initial guess for the lengths of the sides

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
b = (0.0, 60)
bnds = (b, b, b) ## greater than 0 constraints
con1 = {'type': 'ineq', 'fun': triangle_constraint}
cons = ([con1])
solution = minimize(triangle_area, x0, method='SLSQP', bounds=bnds, constraints=cons)
a, b, c = solution.x

area = triangle_area(solution.x) * -1
print(area)

# show final objective
print('Final Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('x1 = ' + str(a))
print('x2 = ' + str(b))
print('x3 = ' + str(c))

Initial Objective: 15.0
173.20508075688707
Final Objective: 179.99999999999912
Solution
x1 = 19.999999999999964
x2 = 19.99999999999996
x3 = 19.99999999999996


In [22]:
result = minimize(triangle_area, x0, method='SLSQP', constraints=[{"type": "ineq", "fun": triangle_constraint}])

a, b, c = result.x
s = (a + b + c) / 2
area = np.sqrt(s * (s - a) * (s - b) * (s - c))

print("The lengths of the sides of the triangle are: a = {}, b = {}, c = {}".format(a, b, c))
print("The area of the triangle is: {}".format(area))

The lengths of the sides of the triangle are: a = 19.999999999999964, b = 19.99999999999996, c = 19.99999999999996
The area of the triangle is: 173.20508075688707
